In [1]:
import tensorflow as tf
import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train

y_train_converted = keras.utils.to_categorical(y_train)
y_test_converted = keras.utils.to_categorical(y_test)

x_train_remolded = x_train.reshape((60000, 784))
x_test_remolded = x_test.reshape((10000, 784))

x_train_normalized = x_train_remolded.astype("float32") / 255
x_test_normalized = x_test_remolded.astype("float32") / 255

In [2]:
learning_rate = 0.001
epochs = 10
batch_size = 200
dropout_rate = 0.3
l2_alpha = 0.0001

In [3]:
n_input = x_train_normalized.shape[1]
n_layer_1 = 30
n_layer_2 = 30
n_class = y_train_converted.shape[1]

In [4]:
w1 = tf.Variable(tf.random.normal([n_input, n_layer_1], stddev=0.05))
b1 = tf.Variable(tf.zeros([n_layer_1]))

w2 = tf.Variable(tf.random.normal([n_layer_1, n_layer_2], stddev=0.05))
b2 = tf.Variable(tf.zeros([n_layer_2]))

w_out = tf.Variable(tf.random.normal([n_layer_2, n_class], stddev=0.05))
b_out = tf.Variable(tf.zeros([n_class]))

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [6]:
def feed_forward(x, training):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))
    #layer_1 = tf.nn.dropout(layer_1, rate=dropout_rate if training else 0.0) dropout
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, w2), b2))
    #layer_2 = tf.nn.dropout(layer_2, rate=dropout_rate if training else 0.0) dropout
    output = tf.matmul(layer_2, w_out) + b_out
    return output

# Função de perda
def loss_fn(logits, labels):
    cross = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    #l2_loss =  l2_alpha*(tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(w_out)) #regularizador l2
    return cross #+ l2_loss

for epoch in range(epochs):
    for batch in range(0, len(x_train_normalized), batch_size):
        x_batch = x_train_normalized[batch:batch + batch_size]
        y_batch = y_train_converted[batch:batch + batch_size]

        with tf.GradientTape() as tape:
            logits = feed_forward(x_batch, training=True)
            loss = loss_fn(logits, y_batch)
        
        # Calcula e aplica os gradientes
        grads = tape.gradient(loss, [w1, b1, w2, b2, w_out, b_out])
        optimizer.apply_gradients(zip(grads, [w1, b1, w2, b2, w_out, b_out]))

    # Cálculo da acurácia (fora do tape)
    logits_train = feed_forward(x_train_normalized, training=False)  # `training=False` desativa Dropout
    correct_prediction_train = tf.equal(tf.argmax(logits_train, 1), tf.argmax(y_train_converted, 1))
    accuracy_train = tf.reduce_mean(tf.cast(correct_prediction_train, tf.float32))

    logits_test = feed_forward(x_test_normalized, training=False)  # `training=False` desativa Dropout
    correct_prediction_test = tf.equal(tf.argmax(logits_test, 1), tf.argmax(y_test_converted, 1))
    accuracy_test = tf.reduce_mean(tf.cast(correct_prediction_test, tf.float32))

    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}, Train Accuracy: {accuracy_train.numpy()}, Test Accuracy: {accuracy_test.numpy()}')

Epoch 1, Loss: 0.2276322990655899, Train Accuracy: 0.8881833553314209, Test Accuracy: 0.8880000114440918
Epoch 2, Loss: 0.1814473271369934, Train Accuracy: 0.9131666421890259, Test Accuracy: 0.9125999808311462
Epoch 3, Loss: 0.1640845239162445, Train Accuracy: 0.9229833483695984, Test Accuracy: 0.9222999811172485
Epoch 4, Loss: 0.15051594376564026, Train Accuracy: 0.9315999746322632, Test Accuracy: 0.9289000034332275
Epoch 5, Loss: 0.1363469362258911, Train Accuracy: 0.9394166469573975, Test Accuracy: 0.9358000159263611
Epoch 6, Loss: 0.12448141723871231, Train Accuracy: 0.9456999897956848, Test Accuracy: 0.9417999982833862
Epoch 7, Loss: 0.1163296326994896, Train Accuracy: 0.9503833055496216, Test Accuracy: 0.9476000070571899
Epoch 8, Loss: 0.11060789227485657, Train Accuracy: 0.9544833302497864, Test Accuracy: 0.9495000243186951
Epoch 9, Loss: 0.10618840903043747, Train Accuracy: 0.9571833610534668, Test Accuracy: 0.9521999955177307
Epoch 10, Loss: 0.1048191636800766, Train Accuracy: